In [7]:
import json
import pandas as pd
import numpy as np
import csv
import random
import matplotlib.pyplot as plt
import seaborn as sns
from joypy import joyplot
from matplotlib.cm import viridis
import plotly.express as px
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset

In [8]:
# import kagglehub
# path = kagglehub.dataset_download("meryentr/text-to-sql")
# print("Path to dataset files:", path)

In [9]:
# import kagglehub
# path = kagglehub.dataset_download("mohammadnouralawad/spider-text-sql")
# print("Path to dataset files:", path)

In [10]:
with open(r"E:\Project_MachineLearning_NguyenMinhVu\Source\Basic-text-to-SQL-conversion\ViText2SQL-master\ViText2SQL-master\data\syllable-level\dev.json", "r", encoding="utf-8") as f1:
    data1 = json.load(f1)
DF_s1 = pd.DataFrame(data1)
with open(r"E:\Project_MachineLearning_NguyenMinhVu\Source\Basic-text-to-SQL-conversion\ViText2SQL-master\ViText2SQL-master\data\syllable-level\tables.json", "r", encoding="utf-8") as f2:
    data2 = json.load(f2)
DF_s2 = pd.DataFrame(data2)
with open(r"E:\Project_MachineLearning_NguyenMinhVu\Source\Basic-text-to-SQL-conversion\ViText2SQL-master\ViText2SQL-master\data\syllable-level\test.json", "r", encoding="utf-8") as f3:
    data3 = json.load(f3)
DF_s3 = pd.DataFrame(data3)
with open(r"E:\Project_MachineLearning_NguyenMinhVu\Source\Basic-text-to-SQL-conversion\ViText2SQL-master\ViText2SQL-master\data\syllable-level\train.json", "r", encoding="utf-8") as f4:
    data4 = json.load(f4)
DF_s4 = pd.DataFrame(data4)
with open(r"E:\Project_MachineLearning_NguyenMinhVu\Source\Basic-text-to-SQL-conversion\ViText2SQL-master\ViText2SQL-master\data\word-level\dev.json", "r", encoding="utf-8") as f5:
    data5 = json.load(f5)
DF_w1 = pd.DataFrame(data5)
with open(r"E:\Project_MachineLearning_NguyenMinhVu\Source\Basic-text-to-SQL-conversion\ViText2SQL-master\ViText2SQL-master\data\word-level\tables.json", "r", encoding="utf-8") as f6:
    data6 = json.load(f6)
DF_w2 = pd.DataFrame(data6)
with open(r"E:\Project_MachineLearning_NguyenMinhVu\Source\Basic-text-to-SQL-conversion\ViText2SQL-master\ViText2SQL-master\data\word-level\test.json", "r", encoding="utf-8") as f7:
    data7 = json.load(f7)
DF_w3 = pd.DataFrame(data7)
with open(r"E:\Project_MachineLearning_NguyenMinhVu\Source\Basic-text-to-SQL-conversion\ViText2SQL-master\ViText2SQL-master\data\word-level\train.json", "r", encoding="utf-8") as f8:
    data8 = json.load(f8)
DF_w4 = pd.DataFrame(data8)


In [11]:
DFSQL2 = pd.read_csv(r"E:\Project_MachineLearning_NguyenMinhVu\Source\Basic-text-to-SQL-conversion\T2SQL2\spider_text_sql.csv")
DFSQL1 = pd.read_csv(r"E:\Project_MachineLearning_NguyenMinhVu\Source\Basic-text-to-SQL-conversion\T2SQL1\train.csv")
DFSQL1

,Unnamed: 0,question,query,schema
0,0,What is the total number of addresses for each...,"SELECT country, state_province_county, COUNT(*...",\nCREATE TABLE addresses (\n\taddress_id INTEG...
1,1,Find all addresses in the United States that d...,SELECT * FROM addresses WHERE country = 'Unite...,\nCREATE TABLE addresses (\n\taddress_id INTEG...
2,2,What is the count of addresses in Europe by ci...,"SELECT city, COUNT(*) AS address_count FROM ad...",\nCREATE TABLE addresses (\n\taddress_id INTEG...
3,3,"Get a list of all countries, along with the nu...","SELECT country, COUNT(*) AS address_count FROM...",\nCREATE TABLE addresses (\n\taddress_id INTEG...
4,4,What is the total number of addresses for each...,"SELECT city, COUNT(*) AS total_addresses FROM ...",\nCREATE TABLE addresses (\n\taddress_id INTEG...
...,...,...,...,...
65349,79952,What are the languages spoken in the country o...,SELECT language FROM countrylanguage WHERE cou...,"\nCREATE TABLE city (\n\tcity_id INTEGER, \n\t..."
65350,79953,What is the city with the highest GDP in the S...,"SELECT city, gdp FROM city WHERE hanzi LIKE 'S...","\nCREATE TABLE city (\n\tcity_id INTEGER, \n\t..."
65351,79955,What is the average population of cities in th...,SELECT AVG(regional_population) FROM city WHER...,"\nCREATE TABLE city (\n\tcity_id INTEGER, \n\t..."
65352,79956,Which country has the highest GDP among countr...,"SELECT country_name, SUM(gdp) AS total_gdp FRO...","\nCREATE TABLE city (\n\tcity_id INTEGER, \n\t..."


In [12]:
DF1 = pd.concat([DF_s1, DF_s3, DF_s4, DF_w1, DF_w3, DF_w4], axis=0)
DF1

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql
0,architecture,select count ( * ) from kiến trúc sư where giớ...,"[select, count, (, *, ), from, kiến trúc sư, w...","[select, count, (, *, ), from, kiến trúc sư, w...",Có tất cả bao nhiêu kiến trúc sư nữ ?,"[Có, tất, cả, bao, nhiêu, kiến, trúc, sư, nữ, ?]","{'except': None, 'from': {'conds': [], 'table_..."
1,architecture,"select tên , quốc tịch , id from kiến trúc sư ...","[select, tên, ,, quốc tịch, ,, id, from, kiến ...","[select, tên, ,, quốc tịch, ,, id, from, kiến ...","Liệt kê tên , quốc tịch và id của tất cả các k...","[Liệt, kê, tên, ,, quốc, tịch, và, id, của, tấ...","{'except': None, 'from': {'conds': [], 'table_..."
2,architecture,"select max ( t1.chiều dài theo mét ) , t2.tên ...","[select, max, (, t1.chiều dài theo mét, ), ,, ...","[select, max, (, t1, ., chiều dài theo mét, ),...",Cho biết tên của người kiến trúc sư đã tạo ra ...,"[Cho, biết, tên, của, người, kiến, trúc, sư, đ...","{'except': None, 'from': {'conds': [[False, 2,..."
3,architecture,select avg ( chiều dài theo feet ) from cầu,"[select, avg, (, chiều dài theo feet, ), from,...","[select, avg, (, chiều dài theo feet, ), from,...",Cho biết chiều dài trung bình tính theo feet c...,"[Cho, biết, chiều, dài, trung, bình, tính, the...","{'except': None, 'from': {'conds': [], 'table_..."
4,architecture,"select tên , năm xây dựng from nhà máy where l...","[select, tên, ,, năm xây dựng, from, nhà máy, ...","[select, tên, ,, năm xây dựng, from, nhà máy, ...",Cho biết tên và năm xây dựng của các nhà máy t...,"[Cho, biết, tên, và, năm, xây, dựng, của, các,...","{'except': None, 'from': {'conds': [], 'table_..."
...,...,...,...,...,...,...,...
6826,wrestler,select đội from lần bị loại where bị loại bởi ...,"[select, đội, from, lần bị loại, where, bị loạ...","[select, đội, from, lần bị loại, where, bị loạ...",Những đội có các đô_vật bị loại bởi ' Orton ' ...,"[Những, đội, có, các, đô_vật, bị, loại, bởi, '...","{'except': None, 'from': {'conds': [], 'table_..."
6827,wrestler,select count ( distinct đội ) from lần bị loại,"[select, count, (, distinct, đội, ), from, lần...","[select, count, (, distinct, đội, ), from, lần...",Những đô_vật đã bị loại đại_diện cho tất_cả ba...,"[Những, đô_vật, đã, bị, loại, đại_diện, cho, t...","{'except': None, 'from': {'conds': [], 'table_..."
6828,wrestler,select count ( distinct đội ) from lần bị loại,"[select, count, (, distinct, đội, ), from, lần...","[select, count, (, distinct, đội, ), from, lần...",Có bao_nhiêu đội khác nhau có thành_viên là cá...,"[Có, bao_nhiêu, đội, khác, nhau, có, thành_viê...","{'except': None, 'from': {'conds': [], 'table_..."
6829,wrestler,select thời_gian from lần bị loại where bị loạ...,"[select, thời_gian, from, lần bị loại, where, ...","[select, thời_gian, from, lần bị loại, where, ...",Những đô_vật đã bị loại bởi ' Punk ' hoặc ' Or...,"[Những, đô_vật, đã, bị, loại, bởi, ', Punk, ',...","{'except': None, 'from': {'conds': [], 'table_..."


In [13]:
DF1.reset_index(level=0, inplace=True)
DF1

,index,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql
0,0,architecture,select count ( * ) from kiến trúc sư where giớ...,"[select, count, (, *, ), from, kiến trúc sư, w...","[select, count, (, *, ), from, kiến trúc sư, w...",Có tất cả bao nhiêu kiến trúc sư nữ ?,"[Có, tất, cả, bao, nhiêu, kiến, trúc, sư, nữ, ?]","{'except': None, 'from': {'conds': [], 'table_..."
1,1,architecture,"select tên , quốc tịch , id from kiến trúc sư ...","[select, tên, ,, quốc tịch, ,, id, from, kiến ...","[select, tên, ,, quốc tịch, ,, id, from, kiến ...","Liệt kê tên , quốc tịch và id của tất cả các k...","[Liệt, kê, tên, ,, quốc, tịch, và, id, của, tấ...","{'except': None, 'from': {'conds': [], 'table_..."
2,2,architecture,"select max ( t1.chiều dài theo mét ) , t2.tên ...","[select, max, (, t1.chiều dài theo mét, ), ,, ...","[select, max, (, t1, ., chiều dài theo mét, ),...",Cho biết tên của người kiến trúc sư đã tạo ra ...,"[Cho, biết, tên, của, người, kiến, trúc, sư, đ...","{'except': None, 'from': {'conds': [[False, 2,..."
3,3,architecture,select avg ( chiều dài theo feet ) from cầu,"[select, avg, (, chiều dài theo feet, ), from,...","[select, avg, (, chiều dài theo feet, ), from,...",Cho biết chiều dài trung bình tính theo feet c...,"[Cho, biết, chiều, dài, trung, bình, tính, the...","{'except': None, 'from': {'conds': [], 'table_..."
4,4,architecture,"select tên , năm xây dựng from nhà máy where l...","[select, tên, ,, năm xây dựng, from, nhà máy, ...","[select, tên, ,, năm xây dựng, from, nhà máy, ...",Cho biết tên và năm xây dựng của các nhà máy t...,"[Cho, biết, tên, và, năm, xây, dựng, của, các,...","{'except': None, 'from': {'conds': [], 'table_..."
...,...,...,...,...,...,...,...,...
19381,6826,wrestler,select đội from lần bị loại where bị loại bởi ...,"[select, đội, from, lần bị loại, where, bị loạ...","[select, đội, from, lần bị loại, where, bị loạ...",Những đội có các đô_vật bị loại bởi ' Orton ' ...,"[Những, đội, có, các, đô_vật, bị, loại, bởi, '...","{'except': None, 'from': {'conds': [], 'table_..."
19382,6827,wrestler,select count ( distinct đội ) from lần bị loại,"[select, count, (, distinct, đội, ), from, lần...","[select, count, (, distinct, đội, ), from, lần...",Những đô_vật đã bị loại đại_diện cho tất_cả ba...,"[Những, đô_vật, đã, bị, loại, đại_diện, cho, t...","{'except': None, 'from': {'conds': [], 'table_..."
19383,6828,wrestler,select count ( distinct đội ) from lần bị loại,"[select, count, (, distinct, đội, ), from, lần...","[select, count, (, distinct, đội, ), from, lần...",Có bao_nhiêu đội khác nhau có thành_viên là cá...,"[Có, bao_nhiêu, đội, khác, nhau, có, thành_viê...","{'except': None, 'from': {'conds': [], 'table_..."
19384,6829,wrestler,select thời_gian from lần bị loại where bị loạ...,"[select, thời_gian, from, lần bị loại, where, ...","[select, thời_gian, from, lần bị loại, where, ...",Những đô_vật đã bị loại bởi ' Punk ' hoặc ' Or...,"[Những, đô_vật, đã, bị, loại, bởi, ', Punk, ',...","{'except': None, 'from': {'conds': [], 'table_..."


In [14]:
DF1.drop(DF1.columns[0], axis=1, inplace=True)
DF1


,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql
0,architecture,select count ( * ) from kiến trúc sư where giớ...,"[select, count, (, *, ), from, kiến trúc sư, w...","[select, count, (, *, ), from, kiến trúc sư, w...",Có tất cả bao nhiêu kiến trúc sư nữ ?,"[Có, tất, cả, bao, nhiêu, kiến, trúc, sư, nữ, ?]","{'except': None, 'from': {'conds': [], 'table_..."
1,architecture,"select tên , quốc tịch , id from kiến trúc sư ...","[select, tên, ,, quốc tịch, ,, id, from, kiến ...","[select, tên, ,, quốc tịch, ,, id, from, kiến ...","Liệt kê tên , quốc tịch và id của tất cả các k...","[Liệt, kê, tên, ,, quốc, tịch, và, id, của, tấ...","{'except': None, 'from': {'conds': [], 'table_..."
2,architecture,"select max ( t1.chiều dài theo mét ) , t2.tên ...","[select, max, (, t1.chiều dài theo mét, ), ,, ...","[select, max, (, t1, ., chiều dài theo mét, ),...",Cho biết tên của người kiến trúc sư đã tạo ra ...,"[Cho, biết, tên, của, người, kiến, trúc, sư, đ...","{'except': None, 'from': {'conds': [[False, 2,..."
3,architecture,select avg ( chiều dài theo feet ) from cầu,"[select, avg, (, chiều dài theo feet, ), from,...","[select, avg, (, chiều dài theo feet, ), from,...",Cho biết chiều dài trung bình tính theo feet c...,"[Cho, biết, chiều, dài, trung, bình, tính, the...","{'except': None, 'from': {'conds': [], 'table_..."
4,architecture,"select tên , năm xây dựng from nhà máy where l...","[select, tên, ,, năm xây dựng, from, nhà máy, ...","[select, tên, ,, năm xây dựng, from, nhà máy, ...",Cho biết tên và năm xây dựng của các nhà máy t...,"[Cho, biết, tên, và, năm, xây, dựng, của, các,...","{'except': None, 'from': {'conds': [], 'table_..."
...,...,...,...,...,...,...,...
19381,wrestler,select đội from lần bị loại where bị loại bởi ...,"[select, đội, from, lần bị loại, where, bị loạ...","[select, đội, from, lần bị loại, where, bị loạ...",Những đội có các đô_vật bị loại bởi ' Orton ' ...,"[Những, đội, có, các, đô_vật, bị, loại, bởi, '...","{'except': None, 'from': {'conds': [], 'table_..."
19382,wrestler,select count ( distinct đội ) from lần bị loại,"[select, count, (, distinct, đội, ), from, lần...","[select, count, (, distinct, đội, ), from, lần...",Những đô_vật đã bị loại đại_diện cho tất_cả ba...,"[Những, đô_vật, đã, bị, loại, đại_diện, cho, t...","{'except': None, 'from': {'conds': [], 'table_..."
19383,wrestler,select count ( distinct đội ) from lần bị loại,"[select, count, (, distinct, đội, ), from, lần...","[select, count, (, distinct, đội, ), from, lần...",Có bao_nhiêu đội khác nhau có thành_viên là cá...,"[Có, bao_nhiêu, đội, khác, nhau, có, thành_viê...","{'except': None, 'from': {'conds': [], 'table_..."
19384,wrestler,select thời_gian from lần bị loại where bị loạ...,"[select, thời_gian, from, lần bị loại, where, ...","[select, thời_gian, from, lần bị loại, where, ...",Những đô_vật đã bị loại bởi ' Punk ' hoặc ' Or...,"[Những, đô_vật, đã, bị, loại, bởi, ', Punk, ',...","{'except': None, 'from': {'conds': [], 'table_..."


In [15]:
DF1[DF1['db_id'].isnull()]

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql


In [16]:
DF1[DF1['query'].isnull()]


,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql


In [17]:
DF1[DF1['query_toks'].isnull()]


,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql


In [18]:
DF1[DF1['query_toks_no_value'].isnull()]


,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql


In [19]:
DF1[DF1['question'].isnull()]


,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql


In [20]:
DF1[DF1['question_toks'].isnull()]


,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql


In [21]:
DF1[DF1['sql'].isnull()]


,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql


In [22]:
DF1.sort_values(by='db_id', ascending=True)

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql
2891,academic,select t3.tiêu đề from viết as t2 join tác giả...,"[select, t3.tiêu đề, from, viết, as, t2, join,...","[select, t3, ., tiêu đề, from, viết, as, t2, j...",những bài báo nào đã được viết bởi H. V. Jagad...,"[những, bài, báo, nào, đã, được, viết, bởi, H....","{'except': None, 'from': {'conds': [[False, 2,..."
3039,academic,select t1.tên from bài báo as t4 join hội nghị...,"[select, t1.tên, from, bài báo, as, t4, join, ...","[select, t1, ., tên, from, bài báo, as, t4, jo...",liệt kê các tác giả có hơn 10 bài báo được đăn...,"[liệt, kê, các, tác, giả, có, hơn, 10, bài, bá...","{'except': None, 'from': {'conds': [[False, 2,..."
3040,academic,select t1.tên from bài báo as t4 join hội nghị...,"[select, t1.tên, from, bài báo, as, t4, join, ...","[select, t1, ., tên, from, bài báo, as, t4, jo...",tác giả nào có nhiều bài báo tại hội nghị VLDB...,"[tác, giả, nào, có, nhiều, bài, báo, tại, hội,...","{'except': None, 'from': {'conds': [[False, 2,..."
3041,academic,select t1.tên from tổ chức as t2 join tác giả ...,"[select, t1.tên, from, tổ chức, as, t2, join, ...","[select, t1, ., tên, from, tổ chức, as, t2, jo...",những tác giả nào thuộc ' Đại học Michigan ' đ...,"[những, tác, giả, nào, thuộc, ', Đại, học, Mic...","{'except': None, 'from': {'conds': [[False, 2,..."
3042,academic,select t1.tên from lĩnh vực của tác giả as t6 ...,"[select, t1.tên, from, lĩnh vực của tác giả, a...","[select, t1, ., tên, from, lĩnh vực của tác gi...",liệt kê các tác giả thuộc ' Đại học Michigan '...,"[liệt, kê, các, tác, giả, thuộc, ', Đại, học, ...","{'except': None, 'from': {'conds': [[False, 2,..."
...,...,...,...,...,...,...,...
12536,yelp,select count ( distinct t1.tên ) from loại_hìn...,"[select, count, (, distinct, t1.tên, ), from, ...","[select, count, (, distinct, t1, ., tên, ), fr...",có bao_nhiêu doanh_nghiệp có loại_hình kinh_do...,"[có, bao_nhiêu, doanh_nghiệp, có, loại_hình, k...","{'except': None, 'from': {'conds': [[False, 2,..."
12535,yelp,select count ( distinct tên ) from người tiêu_...,"[select, count, (, distinct, tên, ), from, ngư...","[select, count, (, distinct, tên, ), from, ngư...",cho biết số_lượng người dùng có tên là Michelle .,"[cho, biết, số_lượng, người, dùng, có, tên, là...","{'except': None, 'from': {'conds': [], 'table_..."
12554,yelp,select t1.tên from loại_hình as t2 join doanh_...,"[select, t1.tên, from, loại_hình, as, t2, join...","[select, t1, ., tên, from, loại_hình, as, t2, ...",doanh_nghiệp nào đang theo nhiều loại_hình kin...,"[doanh_nghiệp, nào, đang, theo, nhiều, loại_hì...","{'except': None, 'from': {'conds': [[False, 2,..."
12540,yelp,select count ( distinct văn_bản ) from đánh_gi...,"[select, count, (, distinct, văn_bản, ), from,...","[select, count, (, distinct, văn_bản, ), from,...",cho biết số_lượng đánh_giá được viết vào tháng...,"[cho, biết, số_lượng, đánh_giá, được, viết, và...","{'except': None, 'from': {'conds': [], 'table_..."


In [23]:
DF_db_id = DF1['db_id']
list_db_id = list(DF_db_id.drop_duplicates())
print(list_db_id)
print(len(list_db_id))

['architecture', 'cinema', 'college_3', 'concert_singer', 'customers_campaigns_ecommerce', 'customers_card_transactions', 'document_management', 'e_learning', 'farm', 'flight_company', 'manufacturer', 'museum_visit', 'network_1', 'party_host', 'phone_market', 'product_catalog', 'scientist_1', 'ship_mission', 'singer', 'sports_competition', 'swimming', 'tracking_share_transactions', 'train_station', 'voter_1', 'wta_1', 'assets_maintenance', 'book_2', 'company_1', 'course_teach', 'decoration_competition', 'election_representative', 'match_season', 'perpetrator', 'station_weather', 'wedding', 'activity_1', 'body_builder', 'candidate_poll', 'city_record', 'college_1', 'cre_Doc_Control_Systems', 'cre_Drama_Workshop_Groups', 'customers_and_invoices', 'customers_and_products_contacts', 'department_management', 'driving_school', 'employee_hire_evaluation', 'flight_2', 'game_injury', 'imdb', 'insurance_and_eClaims', 'local_govt_in_alabama', 'machine_repair', 'mountain_photos', 'music_2', 'produ

In [24]:
DF3 = pd.DataFrame(list_db_id)
DF3

,0
0,architecture
1,cinema
2,college_3
3,concert_singer
4,customers_campaigns_ecommerce
...,...
161,university_basketball
162,voter_2
163,wine_1
164,world_1


In [25]:
EVNtitle = pd.read_csv(r"E:\Project_MachineLearning_NguyenMinhVu\Source\Basic-text-to-SQL-conversion\E-VN title\Chu de English - VNM.txt")
DF2 = pd.DataFrame(EVNtitle)
DF2

,col1,col2
0,architecture,"Kiến trúc, Xây dựng, Công trình"
1,cinema,"Rạp chiếu phim, Điện ảnh, Phim trường"
2,college_3,"Trường cao đẳng 3, Đại học, Học viện"
3,concert_singer,"Ca sĩ biểu diễn, Nghệ sĩ, Nhạc công"
4,customers_campaigns_ecommerce,"Chiến dịch khách hàng thương mại điện tử, Tiếp..."
...,...,...
161,university_basketball,"Bóng rổ đại học, Giải đấu sinh viên, Trận đấu ..."
162,voter_2,"Cử tri 2, Người bỏ phiếu, Quyền bầu cử"
163,wine_1,"Rượu vang 1, Đồ uống có cồn, Nho ủ"
164,world_1,"Thế giới 1, Địa cầu, Toàn cầu hóa"


In [26]:
DF2.reset_index(level=0, inplace=True)

In [27]:
DF2

,index,col1,col2
0,0,architecture,"Kiến trúc, Xây dựng, Công trình"
1,1,cinema,"Rạp chiếu phim, Điện ảnh, Phim trường"
2,2,college_3,"Trường cao đẳng 3, Đại học, Học viện"
3,3,concert_singer,"Ca sĩ biểu diễn, Nghệ sĩ, Nhạc công"
4,4,customers_campaigns_ecommerce,"Chiến dịch khách hàng thương mại điện tử, Tiếp..."
...,...,...,...
161,161,university_basketball,"Bóng rổ đại học, Giải đấu sinh viên, Trận đấu ..."
162,162,voter_2,"Cử tri 2, Người bỏ phiếu, Quyền bầu cử"
163,163,wine_1,"Rượu vang 1, Đồ uống có cồn, Nho ủ"
164,164,world_1,"Thế giới 1, Địa cầu, Toàn cầu hóa"


In [28]:
def remove_numeric(text):
    parts = text.split(",")
    cleaned_parts = []
    for part in parts:
        words = part.split()
        words = [w for w in words if not w.isnumeric()]
        cleaned_parts.append(" ".join(words))
    return ", ".join(cleaned_parts)
DF2['col2'] = DF2['col2'].apply(remove_numeric)

DF2

,index,col1,col2
0,0,architecture,"Kiến trúc, Xây dựng, Công trình"
1,1,cinema,"Rạp chiếu phim, Điện ảnh, Phim trường"
2,2,college_3,"Trường cao đẳng, Đại học, Học viện"
3,3,concert_singer,"Ca sĩ biểu diễn, Nghệ sĩ, Nhạc công"
4,4,customers_campaigns_ecommerce,"Chiến dịch khách hàng thương mại điện tử, Tiếp..."
...,...,...,...
161,161,university_basketball,"Bóng rổ đại học, Giải đấu sinh viên, Trận đấu ..."
162,162,voter_2,"Cử tri, Người bỏ phiếu, Quyền bầu cử"
163,163,wine_1,"Rượu vang, Đồ uống có cồn, Nho ủ"
164,164,world_1,"Thế giới, Địa cầu, Toàn cầu hóa"


In [29]:
DF1.insert(column= 'db_id_vn', loc= 1, value=np.nan)

In [30]:
DF1['db_id_vn'] = DF1['db_id']

In [31]:
result = DF2.set_index('col1')['col2'].to_dict()
result

{'architecture': 'Kiến trúc, Xây dựng, Công trình',
 'cinema': 'Rạp chiếu phim, Điện ảnh, Phim trường',
 'college_3': 'Trường cao đẳng, Đại học, Học viện',
 'concert_singer': 'Ca sĩ biểu diễn, Nghệ sĩ, Nhạc công',
 'customers_campaigns_ecommerce': 'Chiến dịch khách hàng thương mại điện tử, Tiếp thị trực tuyến, Kinh doanh số',
 'customers_card_transactions': 'Giao dịch thẻ khách hàng, Thanh toán điện tử, Thẻ tín dụng',
 'document_management': 'Quản lý tài liệu, Hồ sơ, Lưu trữ dữ liệu',
 'e_learning': 'Học trực tuyến, Giáo dục trực tuyến, Khóa học online',
 'farm': 'Nông trại, Trang trại, Canh tác',
 'flight_company': 'Hãng hàng không, Công ty hàng không, Vận tải hàng không',
 'manufacturer': 'Nhà sản xuất, Doanh nghiệp, Công ty chế tạo',
 'museum_visit': 'Tham quan bảo tàng, Du lịch bảo tàng, Khám phá di sản',
 'network_1': 'Mạng lưới, Hệ thống kết nối, Internet',
 'party_host': 'Chủ trì tiệc, Người tổ chức sự kiện, Chủ tiệc',
 'phone_market': 'Thị trường điện thoại, Kinh doanh di động,

In [32]:
DF1['db_id_vn'] = DF1['db_id_vn'].map(result)

In [33]:
DF1.sort_values(by='db_id', ascending=True)

,db_id,db_id_vn,query,query_toks,query_toks_no_value,question,question_toks,sql
2891,academic,"Học thuật, Giáo dục, Nghiên cứu",select t3.tiêu đề from viết as t2 join tác giả...,"[select, t3.tiêu đề, from, viết, as, t2, join,...","[select, t3, ., tiêu đề, from, viết, as, t2, j...",những bài báo nào đã được viết bởi H. V. Jagad...,"[những, bài, báo, nào, đã, được, viết, bởi, H....","{'except': None, 'from': {'conds': [[False, 2,..."
3039,academic,"Học thuật, Giáo dục, Nghiên cứu",select t1.tên from bài báo as t4 join hội nghị...,"[select, t1.tên, from, bài báo, as, t4, join, ...","[select, t1, ., tên, from, bài báo, as, t4, jo...",liệt kê các tác giả có hơn 10 bài báo được đăn...,"[liệt, kê, các, tác, giả, có, hơn, 10, bài, bá...","{'except': None, 'from': {'conds': [[False, 2,..."
3040,academic,"Học thuật, Giáo dục, Nghiên cứu",select t1.tên from bài báo as t4 join hội nghị...,"[select, t1.tên, from, bài báo, as, t4, join, ...","[select, t1, ., tên, from, bài báo, as, t4, jo...",tác giả nào có nhiều bài báo tại hội nghị VLDB...,"[tác, giả, nào, có, nhiều, bài, báo, tại, hội,...","{'except': None, 'from': {'conds': [[False, 2,..."
3041,academic,"Học thuật, Giáo dục, Nghiên cứu",select t1.tên from tổ chức as t2 join tác giả ...,"[select, t1.tên, from, tổ chức, as, t2, join, ...","[select, t1, ., tên, from, tổ chức, as, t2, jo...",những tác giả nào thuộc ' Đại học Michigan ' đ...,"[những, tác, giả, nào, thuộc, ', Đại, học, Mic...","{'except': None, 'from': {'conds': [[False, 2,..."
3042,academic,"Học thuật, Giáo dục, Nghiên cứu",select t1.tên from lĩnh vực của tác giả as t6 ...,"[select, t1.tên, from, lĩnh vực của tác giả, a...","[select, t1, ., tên, from, lĩnh vực của tác gi...",liệt kê các tác giả thuộc ' Đại học Michigan '...,"[liệt, kê, các, tác, giả, thuộc, ', Đại, học, ...","{'except': None, 'from': {'conds': [[False, 2,..."
...,...,...,...,...,...,...,...,...
12536,yelp,"Trang đánh giá Yelp, Nhận xét nhà hàng, Phản h...",select count ( distinct t1.tên ) from loại_hìn...,"[select, count, (, distinct, t1.tên, ), from, ...","[select, count, (, distinct, t1, ., tên, ), fr...",có bao_nhiêu doanh_nghiệp có loại_hình kinh_do...,"[có, bao_nhiêu, doanh_nghiệp, có, loại_hình, k...","{'except': None, 'from': {'conds': [[False, 2,..."
12535,yelp,"Trang đánh giá Yelp, Nhận xét nhà hàng, Phản h...",select count ( distinct tên ) from người tiêu_...,"[select, count, (, distinct, tên, ), from, ngư...","[select, count, (, distinct, tên, ), from, ngư...",cho biết số_lượng người dùng có tên là Michelle .,"[cho, biết, số_lượng, người, dùng, có, tên, là...","{'except': None, 'from': {'conds': [], 'table_..."
12554,yelp,"Trang đánh giá Yelp, Nhận xét nhà hàng, Phản h...",select t1.tên from loại_hình as t2 join doanh_...,"[select, t1.tên, from, loại_hình, as, t2, join...","[select, t1, ., tên, from, loại_hình, as, t2, ...",doanh_nghiệp nào đang theo nhiều loại_hình kin...,"[doanh_nghiệp, nào, đang, theo, nhiều, loại_hì...","{'except': None, 'from': {'conds': [[False, 2,..."
12540,yelp,"Trang đánh giá Yelp, Nhận xét nhà hàng, Phản h...",select count ( distinct văn_bản ) from đánh_gi...,"[select, count, (, distinct, văn_bản, ), from,...","[select, count, (, distinct, văn_bản, ), from,...",cho biết số_lượng đánh_giá được viết vào tháng...,"[cho, biết, số_lượng, đánh_giá, được, viết, và...","{'except': None, 'from': {'conds': [], 'table_..."


In [34]:
import re
DF1['question'] = DF1['question'].apply(lambda x: re.sub(r'_', ' ', x))
# DF1['question'] = DF1['question'].apply(lambda x: re.sub(r" ,", ",", x))
# DF1['question'] = DF1['question'].apply(lambda x: re.sub(r" \.", ".", x))
# DF1['question'] = DF1['question'].apply(lambda x: re.sub(r"\s*'\s*(.*?)\s*'\s*", r" '\1' ", x).strip())
# DF1['question'] = DF1['question'].apply(lambda x: re.sub(r"\s+\.", ".", x))
# DF1


In [35]:
DF_db_id = DF_db_id.drop_duplicates().sort_values(ascending=True)

In [36]:
DF_db_id = pd.DataFrame(DF_db_id)
DFid = DF_db_id
DFid.insert(column= 'ID', loc= len(DFid.columns), value=range(len(DFid)))
DFid


,db_id,ID
2862,academic,0
1193,activity_1,1
3043,aircraft,2
3089,allergy_1,3
3187,apartment_rentals,4
...,...,...
2721,workshop_paper,161
9533,world_1,162
9653,wrestler,163
892,wta_1,164


In [37]:
mapID = DFid.set_index('db_id')['ID'].to_dict()
mapID

{'academic': 0,
 'activity_1': 1,
 'aircraft': 2,
 'allergy_1': 3,
 'apartment_rentals': 4,
 'architecture': 5,
 'assets_maintenance': 6,
 'baseball_1': 7,
 'battle_death': 8,
 'behavior_monitoring': 9,
 'bike_1': 10,
 'body_builder': 11,
 'book_2': 12,
 'browser_web': 13,
 'candidate_poll': 14,
 'car_1': 15,
 'chinook_1': 16,
 'cinema': 17,
 'city_record': 18,
 'climbing': 19,
 'club_1': 20,
 'coffee_shop': 21,
 'college_1': 22,
 'college_2': 23,
 'college_3': 24,
 'company_1': 25,
 'company_employee': 26,
 'company_office': 27,
 'concert_singer': 28,
 'county_public_safety': 29,
 'course_teach': 30,
 'cre_Doc_Control_Systems': 31,
 'cre_Doc_Template_Mgt': 32,
 'cre_Doc_Tracking_DB': 33,
 'cre_Docs_and_Epenses': 34,
 'cre_Drama_Workshop_Groups': 35,
 'cre_Theme_park': 36,
 'csu_1': 37,
 'culture_company': 38,
 'customer_complaints': 39,
 'customer_deliveries': 40,
 'customers_and_addresses': 41,
 'customers_and_invoices': 42,
 'customers_and_products_contacts': 43,
 'customers_campaig

In [38]:
DF1['db_id'] = DF1['db_id'].map(mapID)
DF1

,db_id,db_id_vn,query,query_toks,query_toks_no_value,question,question_toks,sql
0,5,"Kiến trúc, Xây dựng, Công trình",select count ( * ) from kiến trúc sư where giớ...,"[select, count, (, *, ), from, kiến trúc sư, w...","[select, count, (, *, ), from, kiến trúc sư, w...",Có tất cả bao nhiêu kiến trúc sư nữ ?,"[Có, tất, cả, bao, nhiêu, kiến, trúc, sư, nữ, ?]","{'except': None, 'from': {'conds': [], 'table_..."
1,5,"Kiến trúc, Xây dựng, Công trình","select tên , quốc tịch , id from kiến trúc sư ...","[select, tên, ,, quốc tịch, ,, id, from, kiến ...","[select, tên, ,, quốc tịch, ,, id, from, kiến ...","Liệt kê tên , quốc tịch và id của tất cả các k...","[Liệt, kê, tên, ,, quốc, tịch, và, id, của, tấ...","{'except': None, 'from': {'conds': [], 'table_..."
2,5,"Kiến trúc, Xây dựng, Công trình","select max ( t1.chiều dài theo mét ) , t2.tên ...","[select, max, (, t1.chiều dài theo mét, ), ,, ...","[select, max, (, t1, ., chiều dài theo mét, ),...",Cho biết tên của người kiến trúc sư đã tạo ra ...,"[Cho, biết, tên, của, người, kiến, trúc, sư, đ...","{'except': None, 'from': {'conds': [[False, 2,..."
3,5,"Kiến trúc, Xây dựng, Công trình",select avg ( chiều dài theo feet ) from cầu,"[select, avg, (, chiều dài theo feet, ), from,...","[select, avg, (, chiều dài theo feet, ), from,...",Cho biết chiều dài trung bình tính theo feet c...,"[Cho, biết, chiều, dài, trung, bình, tính, the...","{'except': None, 'from': {'conds': [], 'table_..."
4,5,"Kiến trúc, Xây dựng, Công trình","select tên , năm xây dựng from nhà máy where l...","[select, tên, ,, năm xây dựng, from, nhà máy, ...","[select, tên, ,, năm xây dựng, from, nhà máy, ...",Cho biết tên và năm xây dựng của các nhà máy t...,"[Cho, biết, tên, và, năm, xây, dựng, của, các,...","{'except': None, 'from': {'conds': [], 'table_..."
...,...,...,...,...,...,...,...,...
19381,163,"Đô vật, Vận động viên đấu vật, Thể thao đối kháng",select đội from lần bị loại where bị loại bởi ...,"[select, đội, from, lần bị loại, where, bị loạ...","[select, đội, from, lần bị loại, where, bị loạ...",Những đội có các đô vật bị loại bởi ' Orton ' ...,"[Những, đội, có, các, đô_vật, bị, loại, bởi, '...","{'except': None, 'from': {'conds': [], 'table_..."
19382,163,"Đô vật, Vận động viên đấu vật, Thể thao đối kháng",select count ( distinct đội ) from lần bị loại,"[select, count, (, distinct, đội, ), from, lần...","[select, count, (, distinct, đội, ), from, lần...",Những đô vật đã bị loại đại diện cho tất cả ba...,"[Những, đô_vật, đã, bị, loại, đại_diện, cho, t...","{'except': None, 'from': {'conds': [], 'table_..."
19383,163,"Đô vật, Vận động viên đấu vật, Thể thao đối kháng",select count ( distinct đội ) from lần bị loại,"[select, count, (, distinct, đội, ), from, lần...","[select, count, (, distinct, đội, ), from, lần...",Có bao nhiêu đội khác nhau có thành viên là cá...,"[Có, bao_nhiêu, đội, khác, nhau, có, thành_viê...","{'except': None, 'from': {'conds': [], 'table_..."
19384,163,"Đô vật, Vận động viên đấu vật, Thể thao đối kháng",select thời_gian from lần bị loại where bị loạ...,"[select, thời_gian, from, lần bị loại, where, ...","[select, thời_gian, from, lần bị loại, where, ...",Những đô vật đã bị loại bởi ' Punk ' hoặc ' Or...,"[Những, đô_vật, đã, bị, loại, bởi, ', Punk, ',...","{'except': None, 'from': {'conds': [], 'table_..."


In [39]:
DF_id_vn = DF1['db_id_vn'].drop_duplicates()

lstt = []
for i in DF_id_vn:
    S = i.split(", ")
    for j in S:
        lstt.append(j)
    
lstt = sorted(lstt)
print(lstt)

mydict = {lstt[i]: i for i in range(0, len(lstt))}
print(mydict)

['Album', 'An toàn công cộng quận', 'Ban biên tập tạp chí', 'Ban nhạc cổ điển', 'Buôn bán điện thoại', 'Bài báo', 'Bài bạc', 'Bài hát', 'Bài nghiên cứu hội thảo', 'Báo cáo', 'Báo cáo', 'Báo cáo tin tức', 'Bóng chày', 'Bóng rổ đại học', 'Bóng đá', 'Bóng đá', 'Bơi lặn', 'Bơi lội', 'Bạn đồng hành', 'Bản nhạc', 'Bản tin', 'Bảng xếp hạng phim', 'Bảo hiểm báo cáo tổn thất đầu tiên', 'Bảo hiểm và khiếu nại điện tử', 'Bảo hành sản phẩm', 'Bảo trì thiết bị', 'Bảo trì tài sản', 'Bất động sản', 'Bầu cử', 'Bằng cấp', 'Bệnh dị ứng', 'Bệnh viện', 'Bộ phim', 'Bộ phim', 'Bộ sưu tập sản phẩm', 'Ca sĩ', 'Ca sĩ biểu diễn', 'Canh tác', 'Chia sẻ chương trình', 'Chiến dịch khách hàng thương mại điện tử', 'Chung cư', 'Chuyên gia nghiên cứu', 'Chuyến bay', 'Chuyến bay', 'Chính phủ điện tử', 'Chính quyền địa phương và quy hoạch', 'Chính quyền địa phương ở Alabama', 'Chính sách bảo hiểm', 'Chính trị gia', 'Chương trình', 'Chương trình truyền hình', 'Chương trình âm nhạc', 'Chấn thương khi chơi game', 'Chất lượn

In [40]:
# DF1['db_id_vn'] = DF1['db_id_vn'].apply(lambda x: ", ".join(str(mydict.get(item.strip(), -1)) for item in x.split(", ")))
# DF1

In [41]:
DF1 = DF1.drop_duplicates(subset=['question'], keep='first')
DF1

,db_id,db_id_vn,query,query_toks,query_toks_no_value,question,question_toks,sql
0,5,"Kiến trúc, Xây dựng, Công trình",select count ( * ) from kiến trúc sư where giớ...,"[select, count, (, *, ), from, kiến trúc sư, w...","[select, count, (, *, ), from, kiến trúc sư, w...",Có tất cả bao nhiêu kiến trúc sư nữ ?,"[Có, tất, cả, bao, nhiêu, kiến, trúc, sư, nữ, ?]","{'except': None, 'from': {'conds': [], 'table_..."
1,5,"Kiến trúc, Xây dựng, Công trình","select tên , quốc tịch , id from kiến trúc sư ...","[select, tên, ,, quốc tịch, ,, id, from, kiến ...","[select, tên, ,, quốc tịch, ,, id, from, kiến ...","Liệt kê tên , quốc tịch và id của tất cả các k...","[Liệt, kê, tên, ,, quốc, tịch, và, id, của, tấ...","{'except': None, 'from': {'conds': [], 'table_..."
2,5,"Kiến trúc, Xây dựng, Công trình","select max ( t1.chiều dài theo mét ) , t2.tên ...","[select, max, (, t1.chiều dài theo mét, ), ,, ...","[select, max, (, t1, ., chiều dài theo mét, ),...",Cho biết tên của người kiến trúc sư đã tạo ra ...,"[Cho, biết, tên, của, người, kiến, trúc, sư, đ...","{'except': None, 'from': {'conds': [[False, 2,..."
3,5,"Kiến trúc, Xây dựng, Công trình",select avg ( chiều dài theo feet ) from cầu,"[select, avg, (, chiều dài theo feet, ), from,...","[select, avg, (, chiều dài theo feet, ), from,...",Cho biết chiều dài trung bình tính theo feet c...,"[Cho, biết, chiều, dài, trung, bình, tính, the...","{'except': None, 'from': {'conds': [], 'table_..."
4,5,"Kiến trúc, Xây dựng, Công trình","select tên , năm xây dựng from nhà máy where l...","[select, tên, ,, năm xây dựng, from, nhà máy, ...","[select, tên, ,, năm xây dựng, from, nhà máy, ...",Cho biết tên và năm xây dựng của các nhà máy t...,"[Cho, biết, tên, và, năm, xây, dựng, của, các,...","{'except': None, 'from': {'conds': [], 'table_..."
...,...,...,...,...,...,...,...,...
9688,163,"Đô vật, Vận động viên đấu vật, Thể thao đối kháng",select đội from lần bị loại where bị loại bởi ...,"[select, đội, from, lần bị loại, where, bị loạ...","[select, đội, from, lần bị loại, where, bị loạ...",Những đội có các đô vật bị loại bởi ' Orton ' ...,"[Những, đội, có, các, đô, vật, bị, loại, bởi, ...","{'except': None, 'from': {'conds': [], 'table_..."
9689,163,"Đô vật, Vận động viên đấu vật, Thể thao đối kháng",select count ( distinct đội ) from lần bị loại,"[select, count, (, distinct, đội, ), from, lần...","[select, count, (, distinct, đội, ), from, lần...",Những đô vật đã bị loại đại diện cho tất cả ba...,"[Những, đô, vật, đã, bị, loại, đại, diện, cho,...","{'except': None, 'from': {'conds': [], 'table_..."
9690,163,"Đô vật, Vận động viên đấu vật, Thể thao đối kháng",select count ( distinct đội ) from lần bị loại,"[select, count, (, distinct, đội, ), from, lần...","[select, count, (, distinct, đội, ), from, lần...",Có bao nhiêu đội khác nhau có thành viên là cá...,"[Có, bao, nhiêu, đội, khác, nhau, có, thành, v...","{'except': None, 'from': {'conds': [], 'table_..."
9691,163,"Đô vật, Vận động viên đấu vật, Thể thao đối kháng",select thời gian from lần bị loại where bị loạ...,"[select, thời gian, from, lần bị loại, where, ...","[select, thời gian, from, lần bị loại, where, ...",Những đô vật đã bị loại bởi ' Punk ' hoặc ' Or...,"[Những, đô, vật, đã, bị, loại, bởi, ', Punk, '...","{'except': None, 'from': {'conds': [], 'table_..."


In [42]:
import torch
print(torch.__version__)
print(torch.version.cuda)  
print(torch.backends.cudnn.enabled) 

2.6.0+cu118
11.8
True


In [43]:
DF1

,db_id,db_id_vn,query,query_toks,query_toks_no_value,question,question_toks,sql
0,5,"Kiến trúc, Xây dựng, Công trình",select count ( * ) from kiến trúc sư where giớ...,"[select, count, (, *, ), from, kiến trúc sư, w...","[select, count, (, *, ), from, kiến trúc sư, w...",Có tất cả bao nhiêu kiến trúc sư nữ ?,"[Có, tất, cả, bao, nhiêu, kiến, trúc, sư, nữ, ?]","{'except': None, 'from': {'conds': [], 'table_..."
1,5,"Kiến trúc, Xây dựng, Công trình","select tên , quốc tịch , id from kiến trúc sư ...","[select, tên, ,, quốc tịch, ,, id, from, kiến ...","[select, tên, ,, quốc tịch, ,, id, from, kiến ...","Liệt kê tên , quốc tịch và id của tất cả các k...","[Liệt, kê, tên, ,, quốc, tịch, và, id, của, tấ...","{'except': None, 'from': {'conds': [], 'table_..."
2,5,"Kiến trúc, Xây dựng, Công trình","select max ( t1.chiều dài theo mét ) , t2.tên ...","[select, max, (, t1.chiều dài theo mét, ), ,, ...","[select, max, (, t1, ., chiều dài theo mét, ),...",Cho biết tên của người kiến trúc sư đã tạo ra ...,"[Cho, biết, tên, của, người, kiến, trúc, sư, đ...","{'except': None, 'from': {'conds': [[False, 2,..."
3,5,"Kiến trúc, Xây dựng, Công trình",select avg ( chiều dài theo feet ) from cầu,"[select, avg, (, chiều dài theo feet, ), from,...","[select, avg, (, chiều dài theo feet, ), from,...",Cho biết chiều dài trung bình tính theo feet c...,"[Cho, biết, chiều, dài, trung, bình, tính, the...","{'except': None, 'from': {'conds': [], 'table_..."
4,5,"Kiến trúc, Xây dựng, Công trình","select tên , năm xây dựng from nhà máy where l...","[select, tên, ,, năm xây dựng, from, nhà máy, ...","[select, tên, ,, năm xây dựng, from, nhà máy, ...",Cho biết tên và năm xây dựng của các nhà máy t...,"[Cho, biết, tên, và, năm, xây, dựng, của, các,...","{'except': None, 'from': {'conds': [], 'table_..."
...,...,...,...,...,...,...,...,...
9688,163,"Đô vật, Vận động viên đấu vật, Thể thao đối kháng",select đội from lần bị loại where bị loại bởi ...,"[select, đội, from, lần bị loại, where, bị loạ...","[select, đội, from, lần bị loại, where, bị loạ...",Những đội có các đô vật bị loại bởi ' Orton ' ...,"[Những, đội, có, các, đô, vật, bị, loại, bởi, ...","{'except': None, 'from': {'conds': [], 'table_..."
9689,163,"Đô vật, Vận động viên đấu vật, Thể thao đối kháng",select count ( distinct đội ) from lần bị loại,"[select, count, (, distinct, đội, ), from, lần...","[select, count, (, distinct, đội, ), from, lần...",Những đô vật đã bị loại đại diện cho tất cả ba...,"[Những, đô, vật, đã, bị, loại, đại, diện, cho,...","{'except': None, 'from': {'conds': [], 'table_..."
9690,163,"Đô vật, Vận động viên đấu vật, Thể thao đối kháng",select count ( distinct đội ) from lần bị loại,"[select, count, (, distinct, đội, ), from, lần...","[select, count, (, distinct, đội, ), from, lần...",Có bao nhiêu đội khác nhau có thành viên là cá...,"[Có, bao, nhiêu, đội, khác, nhau, có, thành, v...","{'except': None, 'from': {'conds': [], 'table_..."
9691,163,"Đô vật, Vận động viên đấu vật, Thể thao đối kháng",select thời gian from lần bị loại where bị loạ...,"[select, thời gian, from, lần bị loại, where, ...","[select, thời gian, from, lần bị loại, where, ...",Những đô vật đã bị loại bởi ' Punk ' hoặc ' Or...,"[Những, đô, vật, đã, bị, loại, bởi, ', Punk, '...","{'except': None, 'from': {'conds': [], 'table_..."


Cho biết tên và năm xây dựng của các nhà máy thuộc loại ' Grondzeiler ' ?
Đếm số lượng các giáo sư đã từng dạy lớp học có mã là ' ACCT-211 ' .
Hiển thị những đoạn văn bản được viết trên các đánh giá có điểm xếp hạng là 2.5 .

In [44]:
DF1.insert(column= 'question_E', loc= 6, value=np.nan)
DF1

,db_id,db_id_vn,query,query_toks,query_toks_no_value,question,question_E,question_toks,sql
0,5,"Kiến trúc, Xây dựng, Công trình",select count ( * ) from kiến trúc sư where giớ...,"[select, count, (, *, ), from, kiến trúc sư, w...","[select, count, (, *, ), from, kiến trúc sư, w...",Có tất cả bao nhiêu kiến trúc sư nữ ?,NaN,"[Có, tất, cả, bao, nhiêu, kiến, trúc, sư, nữ, ?]","{'except': None, 'from': {'conds': [], 'table_..."
1,5,"Kiến trúc, Xây dựng, Công trình","select tên , quốc tịch , id from kiến trúc sư ...","[select, tên, ,, quốc tịch, ,, id, from, kiến ...","[select, tên, ,, quốc tịch, ,, id, from, kiến ...","Liệt kê tên , quốc tịch và id của tất cả các k...",NaN,"[Liệt, kê, tên, ,, quốc, tịch, và, id, của, tấ...","{'except': None, 'from': {'conds': [], 'table_..."
2,5,"Kiến trúc, Xây dựng, Công trình","select max ( t1.chiều dài theo mét ) , t2.tên ...","[select, max, (, t1.chiều dài theo mét, ), ,, ...","[select, max, (, t1, ., chiều dài theo mét, ),...",Cho biết tên của người kiến trúc sư đã tạo ra ...,NaN,"[Cho, biết, tên, của, người, kiến, trúc, sư, đ...","{'except': None, 'from': {'conds': [[False, 2,..."
3,5,"Kiến trúc, Xây dựng, Công trình",select avg ( chiều dài theo feet ) from cầu,"[select, avg, (, chiều dài theo feet, ), from,...","[select, avg, (, chiều dài theo feet, ), from,...",Cho biết chiều dài trung bình tính theo feet c...,NaN,"[Cho, biết, chiều, dài, trung, bình, tính, the...","{'except': None, 'from': {'conds': [], 'table_..."
4,5,"Kiến trúc, Xây dựng, Công trình","select tên , năm xây dựng from nhà máy where l...","[select, tên, ,, năm xây dựng, from, nhà máy, ...","[select, tên, ,, năm xây dựng, from, nhà máy, ...",Cho biết tên và năm xây dựng của các nhà máy t...,NaN,"[Cho, biết, tên, và, năm, xây, dựng, của, các,...","{'except': None, 'from': {'conds': [], 'table_..."
...,...,...,...,...,...,...,...,...,...
9688,163,"Đô vật, Vận động viên đấu vật, Thể thao đối kháng",select đội from lần bị loại where bị loại bởi ...,"[select, đội, from, lần bị loại, where, bị loạ...","[select, đội, from, lần bị loại, where, bị loạ...",Những đội có các đô vật bị loại bởi ' Orton ' ...,NaN,"[Những, đội, có, các, đô, vật, bị, loại, bởi, ...","{'except': None, 'from': {'conds': [], 'table_..."
9689,163,"Đô vật, Vận động viên đấu vật, Thể thao đối kháng",select count ( distinct đội ) from lần bị loại,"[select, count, (, distinct, đội, ), from, lần...","[select, count, (, distinct, đội, ), from, lần...",Những đô vật đã bị loại đại diện cho tất cả ba...,NaN,"[Những, đô, vật, đã, bị, loại, đại, diện, cho,...","{'except': None, 'from': {'conds': [], 'table_..."
9690,163,"Đô vật, Vận động viên đấu vật, Thể thao đối kháng",select count ( distinct đội ) from lần bị loại,"[select, count, (, distinct, đội, ), from, lần...","[select, count, (, distinct, đội, ), from, lần...",Có bao nhiêu đội khác nhau có thành viên là cá...,NaN,"[Có, bao, nhiêu, đội, khác, nhau, có, thành, v...","{'except': None, 'from': {'conds': [], 'table_..."
9691,163,"Đô vật, Vận động viên đấu vật, Thể thao đối kháng",select thời gian from lần bị loại where bị loạ...,"[select, thời gian, from, lần bị loại, where, ...","[select, thời gian, from, lần bị loại, where, ...",Những đô vật đã bị loại bởi ' Punk ' hoặc ' Or...,NaN,"[Những, đô, vật, đã, bị, loại, bởi, ', Punk, '...","{'except': None, 'from': {'conds': [], 'table_..."


In [45]:
import pandas as pd
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='vi', target='en')

def translate_text(text):
    try:
        return translator.translate(text)
    except Exception as e:
        print(f"Lỗi khi dịch: {text} -> {e}")
        return text  

# DF1["question_E"] = DF1["question"].astype(str).apply(translate_text)

In [46]:
DFSQL1 = DFSQL1.rename(columns={'question': 'question_E'})
DFSQL2 = DFSQL2.rename(columns={'text_query': 'question_E'})
DFSQL2 = DFSQL2.rename(columns={'sql_command': 'query'})

In [47]:
# DFSQL1.insert(column= 'question_VN', loc= 2, value=np.nan)


import pandas as pd
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='en', target='vi')

def translate_text(text):
    try:
        return translator.translate(text)
    except Exception as e:
        print(f"Lỗi khi dịch: {text} -> {e}")
        return text  

# DFSQL1["question_VN"] = DFSQL1["question_E"].astype(str).apply(translate_text)

In [48]:
DFSQL2.insert(column= 'question_VN', loc= 2, value=np.nan)


import pandas as pd
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='en', target='vi')

def translate_text(text):
    try:
        return translator.translate(text)
    except Exception as e:
        print(f"Lỗi khi dịch: {text} -> {e}")
        return text  

# DFSQL2["question_VN"] = DFSQL2["question_E"].astype(str).apply(translate_text)

In [49]:
DFT1 = pd.read_csv(r"E:\Project_MachineLearning_NguyenMinhVu\Source\Basic-text-to-SQL-conversion\Datasets Train\DF1.csv")
DFT2 = pd.read_csv(r"E:\Project_MachineLearning_NguyenMinhVu\Source\Basic-text-to-SQL-conversion\Datasets Train\DFSQL1.csv")
DFT3 = pd.read_csv(r"E:\Project_MachineLearning_NguyenMinhVu\Source\Basic-text-to-SQL-conversion\Datasets Train\DFSQL2.csv")

DFT1 = DFT1.rename(columns={'question': 'question_VN'})

DFT1 = DFT1[['question_E', 'question_VN', 'query']]
DFT2 = DFT2[['question_E', 'question_VN', 'query']]
DFT3 = DFT3[['question_E', 'question_VN', 'query']]

DF_train = pd.concat([DFT1, DFT2, DFT3], axis=0)
DF_train

,question_E,question_VN,query
0,How many female architects are there?,Có tất cả bao nhiêu kiến trúc sư nữ ?,select count ( * ) from kiến trúc sư where giớ...
1,"List the names, nationality and IDs of all mal...","Liệt kê tên , quốc tịch và id của tất cả các k...","select tên , quốc tịch , id from kiến trúc sư ..."
2,Indicates the name of the architect has create...,Cho biết tên của người kiến trúc sư đã tạo ra ...,"select max ( t1.chiều dài theo mét ) , t2.tên ..."
3,Indicates the average length calculated by fee...,Cho biết chiều dài trung bình tính theo feet c...,select avg ( chiều dài theo feet ) from cầu
4,What is the name and year of construction of f...,Cho biết tên và năm xây dựng của các nhà máy t...,"select tên , năm xây dựng from nhà máy where l..."
...,...,...,...
8029,What are the citizenships that are shared by s...,Các công dân được chia sẻ bởi các ca sĩ với mộ...,SELECT Citizenship FROM singer WHERE Birth_Yea...
8030,How many available features are there in total?,Tổng cộng có bao nhiêu tính năng có sẵn?,SELECT count(*) FROM Other_Available_Features
8031,What is the feature type name of feature AirCon?,Tên loại tính năng của tính năng AirCon là gì?,SELECT T2.feature_type_name FROM Other_Availab...
8032,Show the property type descriptions of propert...,Hiển thị các mô tả loại thuộc tính của các thu...,SELECT T2.property_type_description FROM Prope...


In [50]:
data = [
    {"input": f"{q}", "output": sql} 
    for q, sql in zip(DF_train["question_E"], DF_train["query"])
]
dataset = Dataset.from_pandas(pd.DataFrame(data))
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
def preprocess_function(examples):
    inputs = tokenizer(examples["input"], padding="max_length", truncation=True, max_length=512)
    outputs = tokenizer(examples["output"], padding="max_length", truncation=True, max_length=512)
    inputs["labels"] = outputs["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]
training_args = TrainingArguments(
    output_dir="./t5_sql_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,                 
    num_train_epochs=10,  
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)
# trainer.train()


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/83014 [00:00<?, ? examples/s]

c:\Users\minhv\anaconda3\envs\pytorch-env\lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\minhv\AppData\Local\Temp\ipykernel_24060\630767392.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [51]:
# def generate_sql_debug(question):
#     input_text = f"{question}"
#     inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512)
#     inputs = {key: val.to(device) for key, val in inputs.items()}
#     print("Input tokens:", inputs["input_ids"])
#     outputs = model.generate(**inputs)
#     print("Output tokens:", outputs)
#     return tokenizer.decode(outputs[0], skip_special_tokens=True)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# test_question ="Cho biết tên của các khoa hiện đang làm việc ở tòa nhà 'Mergenthaler'?"
# print(generate_sql_debug(test_question))

In [52]:
import re

question = "Có bao nhiêu công ty tất cả ?"

question = re.sub(r"'", r" ' ", question) 
question = re.sub(r"\(", r"( ", question)  
question = re.sub(r"\)", r" )", question) 
question = re.sub(r"([?!])", r" \1", question)  
question = re.sub(r"(?<=[A-Za-z])\s*,\s*", " , ", question) 
question = re.sub(r"(?<=[^0-9])\s*,\s*", " , ", question) 
question = re.sub(r"(?<=[A-Za-z])\s*\.\s*", " . ", question) 
question = re.sub(r"(?<=[^0-9])\s*\.\s*", " . ", question)  
question = re.sub(r"(?<=\d),(?=\d)", ",", question)  
question = re.sub(r"(?<=\d)\.(?=\d)", ".", question)  
question = re.sub(r"\s+", " ", question).strip()

print(question)
if question in DF_train['question_E'].values:  
    query_result = DF_train.loc[DF_train['question_E'] == question, 'query'].values[0]
    print(query_result)
elif question in DF_train['question_VN'].values:
    query_result = DF_train.loc[DF_train['question_VN'] == question, 'query'].values[0]
    print(query_result)

Có bao nhiêu công ty tất cả ?
select count ( * ) from công ty


In [53]:
# from transformers import AutoTokenizer, AutoModel

# model_name = "t5-small"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name)

# # Lưu mô hình vào thư mục local
# model.save_pretrained("./t5-small")
# tokenizer.save_pretrained("./t5-small")


In [54]:
# import torch
# from transformers import T5Tokenizer, T5ForConditionalGeneration

# # Load model & tokenizer
# MODEL_PATH = "./t5_sql_model"  # Chỉnh sửa để trỏ đến checkpoint tốt nhất
# tokenizer = T5Tokenizer.from_pretrained(MODEL_PATH)
# model = T5ForConditionalGeneration.from_pretrained(MODEL_PATH)

# # Chuyển model sang GPU (nếu có)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)
# model.eval()  # Đưa về chế độ inference, giúp tăng tốc độ

# def generate_sql_debug(question):
#     """Chuyển đổi câu hỏi ngôn ngữ tự nhiên sang SQL."""
#     inputs = tokenizer(question, return_tensors="pt", truncation=True, max_length=256).to(device)

#     with torch.no_grad():  # Tắt gradient để tiết kiệm RAM
#         with torch.cuda.amp.autocast():  # Dùng float16 trên GPU nếu có
#             outputs = model.generate(**inputs, max_length=256)  # Giới hạn output tránh sinh quá dài

#     return tokenizer.decode(outputs[0], skip_special_tokens=True)


# def generate_sql_debug(question):
#     input_text = f"{question}"
#     inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512)
#     inputs = {key: val.to(device) for key, val in inputs.items()}
#     print("Input tokens:", inputs["input_ids"])
#     outputs = model.generate(**inputs)
#     print("Output tokens:", outputs)
#     return tokenizer.decode(outputs[0], skip_special_tokens=True)


# # Test
# test_question = "Cho biết tên của các khoa hiện đang làm việc ở tòa nhà 'Mergenthaler'?"
# print(generate_sql_debug(test_question))


In [57]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

def nl2sql(question, model_path=r"E:\Project_MachineLearning_NguyenMinhVu\Source\Basic-text-to-SQL-conversion\Python\t5_sql_model\checkpoint-28017"):
    """
    Chuyển đổi câu hỏi tự nhiên thành câu lệnh SQL bằng mô hình đã huấn luyện.

    Args:
        question (str): Câu hỏi tự nhiên.
        model_path (str): Đường dẫn đến mô hình đã train.

    Returns:
        str: Câu lệnh SQL tương ứng.
    """
    # Load tokenizer và mô hình
    tokenizer = T5Tokenizer.from_pretrained(model_path)
    model = T5ForConditionalGeneration.from_pretrained(model_path)

    # Tokenize câu hỏi đầu vào
    inputs = tokenizer(question, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Tạo câu lệnh SQL dự đoán
    outputs = model.generate(**inputs, max_length=512)
    sql_query = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return sql_query

# Ví dụ sử dụng
question = "Hiển thị tất cả các địa điểm nơi có một quán ăn đã được mở vào năm 2000 và năm 2010 ."
sql_result = nl2sql(question)
print("SQL Query:", sql_result)


SQL Query: select a im from quán ăn where năm  2000 and năm  2010
